# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True) # Custom dialect overides the excel dialect as default, quote all tells to quote all fields

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): # The code tells if you find blanks still not break 
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATIOn = { 'class':'SimpleStrategy', 'replication_factor':1} """)
except Exception as e:
    print(e)
    

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

We need to select 3 columns artist, song, song_length from the table based on sessionId and itemInsession. 
We will have primary key composing of sessionId as partition key so our data is distributed using sessionId 
and we will have itemInsession as clustering column so our information is sorted nicely for the users.
In order to adhere to NoSQL cassandra rules, we will have to use clustering columns in order in SQL query.

Here is the structure of query:

* "select artist,song,length from song_sessions where sessionId=338 and itemInSession=4"

We will use song_sessions as our table name since we emphasizing on sessions data in songs.

#### Create tables in Cassandra

In [8]:
''' I have included the drop statements of tables that helps me to clean up issues and make the code block reusable '''

prequery = "DROP TABLE IF EXISTS song_sessions"
query = "CREATE TABLE IF NOT EXISTS song_sessions"
query = query + "(sessionId int, itemInSession int,artist text,song text, length float, PRIMARY KEY(sessionId,itemInSession))"

'''Execute drop statement first '''
try:
    session.execute(prequery)
except Exception as e:
    print(e) 

''' Create table '''
try:
    session.execute(query)
except Exception as e:
    print(e) 

#### Lodaing the data in the tables

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        '''Insert the data from the CSV file to Cassandra'''
        
        query = "INSERT INTO song_sessions (sessionId,itemInSession,artist,song,length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
         
        ''' We can either follow this or simply replace line contents with appropriate variables'''
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))       

#### Testing for Query 1

In [10]:
test_query = "select artist,song,length from song_sessions where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(test_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We need to select 3 columns artist, song and users with their firstname and lastname from the table based on userId and sessionId. 
We will have primary key composing of userId and sessionID the composite key will avoid distributingsame user
session data on different nodes, hence the performance will be optimized. 
The itemInSession will be used as the clustering column so our data is sorted as per the requirement


Here is the structure of query:

* SELECT itemInSession, artist, song, firstName, lastName FROM song_artists WHERE userId = 10 AND sessionId = 182

We will use song_library as our table name, since we are including information about users, artists and songs 

#### Create tables in Cassandra

In [11]:
prequery = "DROP TABLE IF EXISTS song_library"
query = "CREATE TABLE IF NOT EXISTS song_library"
query = query + "(userId int,sessionId int, artist text,song text, firstName text,lastName text, ,itemInSession int, PRIMARY KEY((userId,sessionId),itemInSession))"

''' Execute drop statement first '''
try:
    session.execute(prequery)
except Exception as e:
    print(e) 
    
''' Execute drop statement first '''

try:
    session.execute(query)
except Exception as e:
    print(e)                   

#### Lodaing the data in the tables

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        '''Insert the data from the CSV file to Cassandra'''
        query = "INSERT INTO song_library (userId, sessionId,itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ''' We can either follow this or simply replace line contents with appropriate variables'''
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Testing for query 2

In [13]:
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM song_library WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




We need get the first name and lastname by filtering the song name. So we can filter the data using Song however since different users can
be listening to same songs. It is advisable to make composite primary key using UserID column which we make the Primary Key unique.
    
* SELECT firstName, lastName FROM song_users WHERE song = 'All Hands Against His Own

Her since we are dealing with Song and Users data I decided to use table name as song_users.

#### Create tables in Cassandra

In [14]:
prequery = "DROP TABLE IF EXISTS song_users"
query = "CREATE TABLE IF NOT EXISTS song_users"
query = query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY(song, userId))"

try:
    session.execute(prequery)
except Exception as e:
    print(e) 

try:
    session.execute(query)
except Exception as e:
    print(e)   

#### Lodaing the data in the tables

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (line[9], int(line[10]), line[1], line[4], ))

#### Testing for Query 3

In [16]:
rows = session.execute("""SELECT firstName, lastName FROM song_users WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:

session.execute("""DROP TABLE song_sessions""")

session.execute("""DROP TABLE song_users""")

session.execute("""DROP TABLE song_library""")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()